In [9]:
!ls ..

Jyotheeswar  Project1  SummPip	Untitled.ipynb


In [11]:
!head ../SummPip/dataset/multi_news/test.truncate.fix.pun.src.txt -n 1

GOP Eyes Gains As Voters In 11 States Pick Governors Enlarge this image toggle caption Jim Cole / AP Jim Cole / AP Voters in 11 states will pick their governors tonight , and Republicans appear on track to increase their numbers by at least one , with the potential to extend their hold to more than two - thirds of the nation 's top state offices . Eight of the gubernatorial seats up for grabs are now held by Democrats ; three are in Republican hands . Republicans currently hold 29 governorships , Democrats have 20 , and Rhode Island 's Gov. Lincoln Chafee is an Independent . Polls and race analysts suggest that only three of tonight 's contests are considered competitive , all in states where incumbent Democratic governors are n't running again : Montana , New Hampshire and Washington . While those state races remain too close to call , Republicans are expected to wrest the North Carolina governorship from Democratic control , and to easily win GOP - held seats in Utah , North Dakota a

In [1]:
#!pip install PyPDF2

In [2]:
#!pip install spacy
#!python -m spacy download en_core_web_md

## load_documents:
- Input:
    - input_data: list of strings (documents) or list of file paths (strings)
    - input_type: string ('documents' or 'file_paths')
- Output: 
    - list of strings (documents)

## build_sentence_graph:
- Input: 
    - list of strings (documents)
- Output: 
    - 2D list (adjacency list representing the sentence graph)

## spectral_clustering:
- Input: 
    - 2D list (adjacency list representing the sentence graph)
- Output: 
    - list of lists, where each inner list contains spaCy sentence objects (each cluster)

## fit:
- Input: 
    - list of strings (documents)  
    
> No output (updates the self.clusters attribute)

## compress_clusters:
- Input: 
    - list of lists, where each inner list contains spaCy sentence objects (clusters)
- Output: 
    - string (final summary)

## transform:
- Output: 
    - string (final summary)  
    
> No input (uses the self.clusters attribute)

## fit_transform:
- Input: 
    - list of strings (documents)
    - Output: string (final summary)

# Helper Codes

In [16]:
import os
import nltk.data


sent_detector = nltk.data.load('tokenizers/punkt/english.pickle')
# read source file into a list of list:
def read_file(path, file_name, read_lead_only=False, read_first_doc=False):
    f = open(os.path.join(path, file_name),"r")
    lines = f.readlines()
    src_list = []
    tag="story_separator_special_tag"
    for line in lines:
        if read_first_doc:
            line = get_first_doc(line)
            sent_list = truncate_doc(line)
        elif read_lead_only:
            sent_list = read_lead_sentences(line,tag=tag)
        else:
            # remove tag; uncomment below for baseline
            line = line.replace(tag, "")
            # tokenzie line to sentences
            sent_list = sent_detector.tokenize(line.strip())
        src_list.append(sent_list)
    return src_list

## Sentence graph module from SummPIP Impl

In [17]:
# -*- coding: utf-8 -*-
"""
Created on 2020-01-16 10:17 PM

author  : michelle
"""

import numpy as np
import gensim.downloader as api
import spacy
from nltk.corpus import wordnet as wn
from ordered_set import OrderedSet
import scipy
from scipy import *
import re

glove_word_vectors = api.load("glove-wiki-gigaword-100")
spacynlp=spacy.load("en_core_web_sm")

# Step 1: Deverbal Noun Reference
# step 1.1: get nouns for verbs in the current sentence
verbs_to_escape = ["be", "is","am","are","was", "were", "being","been","do","did",
               "done","have","had","get","got","gotten"]

# Step 3: Discourse Markers => only for two adjacent sentences
markers=["for","so","because","since","therefore","consequently","additionally","furthermore","moreover",
         "but","however","although","despite","similarly","otherwise","whereas","while","unlike","thus",
        "instead","nevertheless","afterward","finally","subsequently","conversely","later","next","then",
         "likewise","compared","besides","further","as","also","equally","hence","accordingly","stil",
        "simultaneously"]
# 39 markers

class SentenceGraph:
    def __init__(self, sentences_list, w2v, use_lm, lm_model, lm_tokenizer, ita=0.9, threshold=0.65):
        self.sentences_list = sentences_list

        self.length = len(sentences_list)

        self.w2v = w2v

        self.use_lm = use_lm 

        self.lm_model = lm_model

        self.tokenizer = lm_tokenizer

        # threshold for step1
        self.threshold = threshold

        # threshold for step4
        self.ita = ita

    def get_nouns_for_verbs(self, string):
        doc = spacynlp(string)
        nouns_list = []
        if len(doc)>0:
            for token in doc:
                # find noun reference for verbs, escaping verbs that are too ambiguous
                if token.pos_ == "VERB" and token.text not in verbs_to_escape:
                    # print("token.text ", token.text)
                    noun_forms = self._nounify(token.text)
                    nouns_list.extend(noun_forms)
        return nouns_list

    def _nounify(self, verb):
        # get the lemmas of base verbs;
        base = wn.morphy(verb, wn.VERB)
        if base:
            lemmas = wn.lemmas(base, pos="v")
            noun_forms = []
            # derive noun forms for each lemma
            for lemma in lemmas:
                nouns = [forms.name() for forms in lemma.derivationally_related_forms()]
                noun_forms.extend(nouns)
            # remove repetition
            nouns_set = OrderedSet(noun_forms)
            return nouns_set
        else:
            return []

    # step 1.2: find most similar word from word2vec
    # get most similar words for nouns, including itself
    def find_most_similar_words(self, word_vectors,nouns_list,threshold=0.65):
        similar_nouns_list=[]
        nouns_list=list(set(nouns_list))
        for noun in nouns_list:
            try:
                nn = word_vectors.most_similar(positive=[noun])
                # keep nn whose have high similary score
                nn = [ tuple_[0] for tuple_ in nn if tuple_[1] > threshold]
                similar_nouns_list.extend(nn)
            # pass on uncommon words
            except KeyError:
                pass
        similar_nouns_list.extend(nouns_list)
        return list(set(similar_nouns_list))

    # check if deverbal noun reference exits in the subsequent sentence
    def check_noun_reference(self, similar_nouns_list, subsequent_sen):
        flag=False
        doc = spacynlp(subsequent_sen)
        if len(doc)>0:
            for token in doc:
                if token.pos_ == "NOUN":
                    if token.text in similar_nouns_list:
                        flag=True
                        break
        return flag

    # step 2: Event/Entity Continuation
    # Str needs to be raw, i.e., use str before normalisation and stemming
    def compare_name_entity(self, str1, str2):
        flag = False
        doc1 = spacynlp(str1)
        doc2 = spacynlp(str2)
        if len(doc1)>0 and len(doc2)>0:
            ent_list1=[(ent.text, ent.label_) for ent in doc1.ents]
            ent_list2=[(ent.text, ent.label_) for ent in doc2.ents]
            for (text, label) in ent_list1:
                if (text, label) in ent_list2:
                    flag=True
                    break
        return flag

    def check_discourse_markers(self, str1,str2):
        flag = False
        doc2 = spacynlp(str2)
        if len(doc2)>0:
            first_token = doc2[0].text
            if first_token.lower() in markers:
                flag = True
        return flag

    # compute the cos similarity between a and b. a, b are numpy arrays
    def cos_sim(self, a, b):
        return 1 - scipy.spatial.distance.cosine(a,b)


    def make_graph_undirected(self, source, target, weight):
        source.extend(target)
        target.extend(source)
        weight.extend(weight)
        triplet_list=[ (source[i],target[i],weight[i]) for i in range(len(source))]
        sorted_by_src = sorted(triplet_list, key=lambda x: (x[0],x[1]))

        sorted_source = []
        sorted_target = []
        sorted_weight = []
        for triplet in sorted_by_src:
            sorted_source.append(triplet[0])
            sorted_target.append(triplet[1])
            sorted_weight.append(triplet[2])
        return sorted_source, sorted_target, sorted_weight

    # Step4: calculate sentence embeddings
    def get_sentence_embeddings(self,string):
        if not self.use_lm:
            v = self.get_wv_embedding(string)
        else:
            v = self.get_lm_embedding(string)
        return v

    # get sentence embeddings with w2v
    def get_wv_embedding(self, string):
        word_embeddings = self.w2v
        sent = string.lower()
        eps = 1e-10
        if len(sent) != 0:
            vectors = [word_embeddings.get(w, np.zeros((100,))) for w in sent.split()]
            v = np.mean(vectors, axis=0)
        else:
            v = np.zeros((100,))
        v = v + eps
        return v

    # get language model embedding
    def get_lm_embedding(self, string):
        sent = string.lower()
        eps = 1e-10
        if len(sent)!= 0:
            input_ids = torch.tensor([self.tokenizer.encode(sent)])
            last_hidden_state = self.lm_model(input_ids)[0]
            hidden_state=last_hidden_state.tolist()
            v = np.mean(hidden_state,axis=1)
        else:
            v = np.zeros((768,))
        v = v + eps
        return v

    # step 4: compare sentence similarity
    def check_if_similar_sentences(self,sentence_emb1,sentence_emb2):
        flag = False
        similarity = self.cos_sim(sentence_emb1,sentence_emb2)
        if similarity > self.ita:
            flag = True
        return flag


    def build_sentence_graph(self,):
        # spectral clustering  
        X = np.zeros([self.length, self.length])
        
        # get the vector size
        self.size = len(self.get_sentence_embeddings(self.sentences_list[0]))

        # get sentence vector holder
        emb_sentence_vectors = np.zeros([self.length,self.size])
 
        for i in range(self.length):
             emb_sen = self.get_sentence_embeddings(self.sentences_list[i])
             emb_sentence_vectors[i,] = emb_sen
        
        # iterate all sentence nodes to check if they should be connected
        for i in range(self.length):
            flag = False
            sen_i = self.sentences_list[i]
            # check above steps
            for j in range(i+1,self.length):
                sen_j = self.sentences_list[j]
                if (j-i) == 1:
                    # perform step1 and step3,which are only for adjacent sentences
                    nouns_list = self.get_nouns_for_verbs(sen_i)
                    # get most similar words for above nouns, including itself
                    similar_nouns_list = self.find_most_similar_words(glove_word_vectors, nouns_list,self.threshold)
                    # check for devebal noun 
                    flag = self.check_noun_reference(similar_nouns_list, sen_j)
                    if not flag:
                        # check for disourse markers
                        flag=self.check_discourse_markers(sen_i,sen_j)
                else:
                    # check for name entities
                    flag=self.compare_name_entity(sen_i,sen_j)

               # => step4 check for similar sentences
                if not flag:
                    # continue
                    i_sen_emb = emb_sentence_vectors[i,]
                    j_sen_emb = emb_sentence_vectors[j,]
                    flag = self.check_if_similar_sentences(i_sen_emb,j_sen_emb)

                if flag:                                      
                    X[i,j] = 1
                    X[j,i] = 1
        return X


In [143]:
import PyPDF2
import spacy
from sklearn.cluster import SpectralClustering
from sentence_transformers import SentenceTransformer, util
from nltk.parse.earleychart import FeatureEarleyChartParser
from tqdm import tqdm
import configparser
import openai

class SummPIPX:
    def __init__(self, clustering_method='spectral_clustering', graph_method='build_sentence_graph', compression_method='compress_clusters', nb_clusters=14, seed=88):
        self.version = 'v1'
        self.clustering_method = clustering_method
        self.graph_method = graph_method
        self.compression_method = compression_method
        self.nb_clusters = nb_clusters
        self.seed = seed

    def __pdf_to_text__(self, path):
        pdfreader = PyPDF2.PdfReader(path)
        text=''
        for page in pdfreader.pages:
            text+=page.extract_text()
        return text
    
    def load_documents(self, input_data, input_type='documents'):
        # Check the input type
        if input_type == 'documents':
            # If input_type is 'documents', assume input_data represents the documents
            documents = input_data
        elif input_type == 'file_paths':
            # If input_type is 'file_paths', assume input_data represents file paths
            documents = [self.__pdf_to_text__(file) for file in files]
        else:
            raise ValueError(f"Invalid input_type: {input_type}")
        self.documents = documents

    def build_sentence_graph(self):
        # Default graph building method
        nlp = spacy.load('en_core_web_md')
        model = SentenceTransformer('distilbert-base-nli-mean-tokens')

        def discourse_markers(sentence1, sentence2):
            discourse_markers = ["accordingly", "additionally", "although", "as a result", "as an illustration", "as well as", "besides", "consequently", "conversely", "finally", "furthermore", "however", "in addition", "in contrast", "in fact", "in other words", "in short", "in summary", "indeed", "likewise", "meanwhile", "moreover", "namely", "nevertheless", "nonetheless", "on the contrary", "on the other hand", "otherwise", "overall", "similarly", "so", "subsequently", "that is", "then", "thereby", "therefore", "thus", "to clarify", "to demonstrate", "to elaborate", "to emphasize", "to enumerate", "to explain", "to illustrate", "to list", "to point out", "to put it another way", "to put it simply", "to sum up", "to summarize", "ultimately", "while"]
            pattern = r"\b(" + "|".join(discourse_markers) + r")\b"
            if re.search(pattern, sentence2.lower()):
                return True
            else:
                return False

        def sentence_similarity(sentence1, sentence2):
            sentence_embeddings = model.encode([sentence1, sentence2])
            score = util.pytorch_cos_sim(sentence_embeddings[0], sentence_embeddings[1]).item()
            return score > 0.85

        def name_entity_check(sentence1, sentence2):
            doc1 = nlp(sentence1)
            doc2 = nlp(sentence2)
            entities1 = set((e.text, e.label_) for e in doc1.ents)
            entities2 = set((e.text, e.label_) for e in doc2.ents)
            return any(entity in entities1 for entity in entities2)

        def deverbal_noun(sentence1, sentence2, threshold=0.7):
            doc1, doc2 = nlp(sentence1), nlp(sentence2)

            verbs_in_sentence1 = [token.text for token in doc1 if token.pos_ == 'VERB' and token.text not in verbs_to_escape]

            noun_set = set()
            for verb in verbs_in_sentence1:
                base_form = wn.morphy(verb, wn.VERB)
                if base_form:
                    lemmas = wn.lemmas(base_form, pos='v')
                    noun_set |= set(forms.name() for lemma in lemmas for forms in lemma.derivationally_related_forms())
            nouns_in_sentence1 = list(noun_set)

            similar_nouns = set()
            for noun in nouns_in_sentence1:
                if noun in glove_word_vectors:
                    similar_nouns |= set([word for word, score in glove_word_vectors.similar_by_word(noun, topn=10) if score > threshold])
            similar_nouns= list(similar_nouns | set(nouns_in_sentence1))

            check = any(token.pos_ == 'NOUN' and token.text in similar_nouns for token in doc2)

            return check

        sentences = []
        for doc in self.documents:
            sentences.extend(doc)

        num_sentences = len(sentences)
        graph = np.full((num_sentences, num_sentences), False)

        for ind1, sentence1 in enumerate(tqdm(sentences)):
            for ind2 in range(ind1 + 1, len(sentences)):
                sentence2 = sentences[ind2]

                edge = False
                if ind1 + 1 == ind2:
                    edge = discourse_markers(sentence1, sentence2)
                    if not edge:
                        edge = deverbal_noun(sentence1, sentence2)
                if not edge:
                    edge = name_entity_check(sentence1, sentence2)
                if not edge:
                    edge = sentence_similarity(sentence1, sentence2)

                graph[ind1][ind2] = edge
                graph[ind2][ind1] = edge

        return graph.astype(int)
    
    def default_graph_method(self, documents):
        """
        This is the method used in the original implementaion
        """
        sentences = []
        for doc in self.documents:
            sentences.extend(doc)
        SG = SentenceGraph(sentences, _get_w2v_embeddings('word_vec/multi_news/news_w2v.txt'), False, '', '')
        graph = SG.build_sentence_graph()

    def spectral_clustering(self, sentence_graph, documents):
        sentences = []
        for doc in self.documents:
            sentences.extend(doc)
        clustering = SpectralClustering(n_clusters = self.nb_clusters, random_state = self.seed).fit(sentence_graph)
        clusterIDs = clustering.labels_

        num_clusters = max(clusterIDs)+1
        cluster_dict={new_list:[] for new_list in range(num_clusters)}
		# group sentences by cluster ID
        for i, clusterID in enumerate(clusterIDs):
            cluster_dict[clusterID].append(sentences[i])
        return cluster_dict

    def another_clustering_method(self, sentence_graph):
        # Another clustering method
        return []

    def compress_clusters(self, clusters):
        nlp = spacy.load("en_core_web_md")
        summary_sentences = []

        for key in clusters:
            cluster=clusters[key]
            if len(cluster) == 1:
                # If there's only one sentence in the cluster, add it to the summary
                summary_sentences.append(cluster[0])
            else:
                # Convert the sentences in the cluster into Spacy Doc objects
                cluster_docs = [nlp(sent) for sent in cluster]

                # Calculate the similarity scores between all sentences in the cluster
                similarity_matrix = [[sent1.similarity(sent2) for sent2 in cluster_docs] for sent1 in cluster_docs]

                # Calculate the sum of similarity scores for each sentence
                similarity_sums = [sum(row) for row in similarity_matrix]

                # Find the index of the sentence with the highest similarity score sum
                most_relevant_index = similarity_sums.index(max(similarity_sums))

                # Add the most relevant sentence to the summary
                summary_sentences.append(cluster[most_relevant_index])

        # Concatenate the summary sentences
        summary = " ".join(summary_sentences)
        return summary

    def gpt_compression_method(self, clusters):
        config = configparser.ConfigParser()
        config.read('config.ini')
        API_KEY = config.get("openai", "api_key")
        openai.api_key = API_KEY
        summarized_clusters = []

        for key in clusters:
            cluster = clusters[key]
            messages = [{"role": "system", "content": "You are an AI language model. Summarize the following sentences into one coherent sentence."}]
            for sentence in cluster:
                messages.append({"role": "user", "content": sentence})

            response = openai.ChatCompletion.create(
                model="gpt-3.5-turbo",
                messages=messages,
                max_tokens=50,
                n=1,
                stop=None,
                temperature=0.5,
            )

            summary = response.choices[0]['message']['content'].strip()
            summarized_clusters.append(summary)

        final_summary_messages = [{"role": "system", "content": "You are an AI language model. Create a coherent summary of the following summarized sentences."}]
        for summary in summarized_clusters:
            final_summary_messages.append({"role": "user", "content": summary})

        final_summary_response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=final_summary_messages,
            max_tokens=50,
            n=1,
            stop=None,
            temperature=0.5,
        )

        final_summary = final_summary_response.choices[0]['message']['content'].strip()
        return final_summary    


    def fit(self):
        # Call the appropriate graph building method
        graph_method = getattr(self, self.graph_method)
        sentence_graph = graph_method()
        
        # Call the appropriate clustering method
        clustering_method = getattr(self, self.clustering_method)
        self.clusters = clustering_method(sentence_graph, self.documents)

    def transform(self):
        # Call the appropriate compression method
        compression_method = getattr(self, self.compression_method)
        summary = compression_method(self.clusters)
        return summary

## Testing Graph

In [118]:
src_list = read_file('dataset/multi_news/','test.truncate.fix.pun.src.txt')[:2]

In [101]:
summpipx = SummPIPX()
summpipx.load_documents(src_list)
graph_own = summpipx.build_sentence_graph()

100%|██████████| 37/37 [00:18<00:00,  2.04it/s]


In [102]:
def _get_w2v_embeddings(w2v_file):
    """
    Get w2v word embedding matrix

    :return: w2v matrix
    """
    word_embeddings = {}
    f = open(w2v_file, encoding='utf-8')
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        word_embeddings[word] = coefs
    f.close()
    return word_embeddings

all_sents = src_list[0] + src_list[1]
SG = SentenceGraph(all_sents, _get_w2v_embeddings('word_vec/multi_news/news_w2v.txt'), False, '', '')
graph_def = SG.build_sentence_graph()

In [103]:
graph_def

array([[0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.]])

In [104]:
graph_own

array([[0, 0, 1, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 0, 0],
       [1, 1, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [105]:
comp = np.equal(graph_def, graph_own)
print(comp.size, np.sum(comp))

1369 1313


## We can see that we have a very similar graph to the original implementation

In [106]:
clusters=summpipx.spectral_clustering(graph_def, all_sents)
summpipx.compress_clusters(clusters)

/home/vjspranav/jupyter/Project1/summpipenv/lib/python3.10/site-packages/sklearn/cluster/_spectral.py:717: UserWarning: The spectral clustering API has changed. ``fit``now constructs an affinity matrix from data. To use a custom affinity matrix, set ``affinity=precomputed``.
  warnings.warn(


'This picture has itself caused scandal in the UK , as it was a gay kiss that was broadcast before the watershed , and as such led to a number of complaints to the BBC . Before it goes too far , I just want people to know that FB have NOT removed the kiss - in event page ; it ’s still there , but _ I made the  || News || Page 1 of 1 UPDATED : A photo of two men kissing that was posted on a Facebook page protesting a London pub ’s decision to eject a same - sex couple for kissing has been removed by the social networking site , an error , according to a rep for the company . " GOP Eyes Gains As Voters In 11 States Pick Governors Enlarge this image toggle caption Jim Cole / AP Jim Cole / AP Voters in 11 states will pick their governors tonight , and Republicans appear on track to increase their numbers by at least one , with the potential to extend their hold to more than two - thirds of the nation \'s top state offices . UPDATE : 4/19/2001 Read Richard Metzger : How I , a married , midd

# Testing Model

In [144]:
summpipx = SummPIPX()
summpipx.load_documents(src_list)
summpipx.fit()
summpipx.transform()

100%|██████████| 37/37 [00:18<00:00,  2.03it/s]
/home/vjspranav/jupyter/Project1/summpipenv/lib/python3.10/site-packages/sklearn/cluster/_spectral.py:717: UserWarning: The spectral clustering API has changed. ``fit``now constructs an affinity matrix from data. To use a custom affinity matrix, set ``affinity=precomputed``.
  warnings.warn(


'No matter who wins the While those state races remain too close to call , Republicans are expected to wrest the North Carolina governorship from Democratic control , and to easily win GOP - held seats in Utah , North Dakota and Indiana . This picture has itself caused scandal in the UK , as it was a gay kiss that was broadcast before the watershed , and as such led to a number of complaints to the BBC . Before it goes too far , I just want people to know that FB have NOT removed the kiss - in event page ; it ’s still there , but _ I made the  || News || Page 1 of 1 UPDATED : A photo of two men kissing that was posted on a Facebook page protesting a London pub ’s decision to eject a same - sex couple for kissing has been removed by the social networking site , an error , according to a rep for the company . " Eight of the gubernatorial seats up for grabs today are now held by Democrats ; three are in Republican hands . Republicans currently hold 29 governorships , Democrats have 20 , a

In [145]:
summpipxgpt = SummPIPX(compression_method='gpt_compression_method')
summpipxgpt.load_documents(src_list)
summpipxgpt.fit()
summpipxgpt.transform()

100%|██████████| 37/37 [00:18<00:00,  2.01it/s]
/home/vjspranav/jupyter/Project1/summpipenv/lib/python3.10/site-packages/sklearn/cluster/_spectral.py:717: UserWarning: The spectral clustering API has changed. ``fit``now constructs an affinity matrix from data. To use a custom affinity matrix, set ``affinity=precomputed``.
  warnings.warn(


'NYULocal.com reported that a photo was removed from a Facebook page promoting a "gay kiss-in" demonstration in London, and administrators were sent an email stating that nudity and sexually suggestive content is not allowed on the platform. However, the photo was later clarified to not have been removed and was instead made private by the event creator. The photo used in the post was a promotional still from the British soap opera "Eastenders" and was chosen because it was considered mild. Meanwhile, in the current gubernatorial election, Republicans are predicted to take control of the North Carolina governorship from the Democrats and win GOP-held seats in Utah, North Dakota, and Indiana, despite the uncertainty surrounding the other state races.'

# The above summary is generated with 500 tokens and in code it has been hardcoded to 50 to ensure that uneeded tokens don't get used up